In [62]:
import xml.etree.ElementTree as ET
from fuzzywuzzy import fuzz
import json
import xmltodict

In [63]:
def if_same_person(first, second):
    ser = fuzz.token_set_ratio(first, second)    
    return ser > 75

In [64]:
def possibles(fn, ln):
    ret = [ln+"_"+fn, ln+"_"+fn[0], fn+"_"+ln]
    return ret

In [65]:
def combines(a_first_name, a_last_name, b_first_name, b_last_name):
    cl1 = possibles(a_first_name, a_last_name)
    cl2 = possibles(b_first_name, b_last_name)
    ret = []
    for c1 in cl1:
        for c2 in cl2:
            ret.append((c1, c2))
    return ret

In [66]:
def build_url(n1, n2):
    url = "http://export.arxiv.org/api/query?search_query=au:"
    url += n1 +" AND " + "au:"+n2 + "&max_results=1000"
    return url

In [67]:
def get_list(src, p1, p2):
    ret = []
    try:
        for entry in src['feed'][u'entry']:
            unit = {}
            unit['title'] = entry['title']
            unit['authors'] = entry['author']

            score = 0
            for author in unit['authors']:
                if if_same_person(p1, author['name']) or if_same_person(p2, author['name']):
                    score += 1 
            unit['id'] = entry['id']
            if score>=2:
                ret.append(unit)
    except:
        pass
    return ret

In [68]:
def get_papers(a_first_name, a_last_name, b_first_name, b_last_name):
    comb = combines(a_first_name, a_last_name, b_first_name, b_last_name)
    print comb
    col = {}
    for c in comb:
        url = build_url(c[0], c[1])
        print url
        data = urllib.urlopen(url).read()
        hold = json.loads(json.dumps(xmltodict.parse(data)))
        papers = get_list(hold, a_first_name+" "+a_last_name, b_first_name+" "+b_last_name)
        for paper in papers:
            idea = paper['id']
            col[idea] = paper
    ret = []
    for p in col.values():
        ret.append(p['title'])
    return ret

In [70]:
get_papers("Steven", "Bradlow", "Peter", "Gothen")

[('Bradlow_Steven', 'Gothen_Peter'), ('Bradlow_Steven', 'Gothen_P'), ('Bradlow_Steven', 'Peter_Gothen'), ('Bradlow_S', 'Gothen_Peter'), ('Bradlow_S', 'Gothen_P'), ('Bradlow_S', 'Peter_Gothen'), ('Steven_Bradlow', 'Gothen_Peter'), ('Steven_Bradlow', 'Gothen_P'), ('Steven_Bradlow', 'Peter_Gothen')]
http://export.arxiv.org/api/query?search_query=au:Bradlow_Steven AND au:Gothen_Peter&max_results=1000
http://export.arxiv.org/api/query?search_query=au:Bradlow_Steven AND au:Gothen_P&max_results=1000
http://export.arxiv.org/api/query?search_query=au:Bradlow_Steven AND au:Peter_Gothen&max_results=1000
http://export.arxiv.org/api/query?search_query=au:Bradlow_S AND au:Gothen_Peter&max_results=1000
http://export.arxiv.org/api/query?search_query=au:Bradlow_S AND au:Gothen_P&max_results=1000
http://export.arxiv.org/api/query?search_query=au:Bradlow_S AND au:Peter_Gothen&max_results=1000
http://export.arxiv.org/api/query?search_query=au:Steven_Bradlow AND au:Gothen_Peter&max_results=1000
http://expo

[u'Maximal surface group representations in isometry groups of classical\n  Hermitian symmetric spaces',
 u'Deformations of maximal representations in Sp(4,R)',
 u'Representations of the fundamental group of a surface in PU(p,q) and\n  holomorphic triples',
 u'Moduli spaces of holomorphic triples over compact Riemann surfaces',
 u'Representations of surface groups in the general linear group',
 u'Higgs bundles for the non-compact dual of the special orthogonal group',
 u'Surface group representations in PU(p,q) and Higgs bundles',
 u'Surface group representations, Higgs bundles, and holomorphic triples',
 u'Homotopy groups of moduli spaces of representations']